# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [42]:
!pip install --upgrade datasets
!pip install -q "datasets==2.15.0" transformers peft pandas numpy scikit-learn


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://nexus.corp.indeed.com/repository/pypi/simple
  Using cached https://nexus.corp.indeed.com/repository/pypi/packages/datasets/3.2.0/datasets-3.2.0-py3-none-any.whl (480 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.15.0
    Uninstalling datasets-2.15.0:
      Successfully uninstalled datasets-2.15.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [48]:
import torch
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModelForSequenceClassification
from datasets import load_dataset
import pandas as pd
import numpy as np

splits = ["train", "test"]
ds = load_dataset("SetFit/TREC-QC")

tokenized_dataset = {}
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
for split in splits:
    tokenized_dataset[split] = ds[split].map(
        lambda x: tokenizer(x["text"], truncation=True, padding=True), batched=True
    )

tokenized_dataset["test"]



Repo card metadata block was not found. Setting CardData to empty.
Map: 100%|██████████| 500/500 [00:00<00:00, 45273.35 examples/s]


Dataset({
    features: ['text', 'label', 'label_text', 'label_original', 'label_coarse', 'label_coarse_text', 'label_coarse_original', 'input_ids', 'attention_mask'],
    num_rows: 500
})

Dataset({
    features: ['text', 'label', 'label_text', 'label_original', 'label_coarse', 'label_coarse_text', 'label_coarse_original', 'input_ids', 'attention_mask'],
    num_rows: 500
})

In [54]:
# Create model
model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=6,
    id2label={
        0: "ABBR",
        1: "ENTY",
        2: "DESC",
        3: "HUM",
        4: "LOC",
        5: "NUM",
    },
    label2id={
        "ABBR": 0,
        "ENTY": 1,
        "DESC": 2,
        "HUM": 3,
        "LOC": 4,
        "NUM": 5,
    }
)

# Freeze all the parameters of the base model
for param in model.base_model.parameters():
    param.requires_grad = False

model.config.pad_token_id = model.config.eos_token_id
print(model)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=6, bias=False)
)


In [60]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, EvalPrediction

# Function to compute metrics
def compute_metrics(eval_pred: EvalPrediction):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(eval_pred.label_ids, predictions, average='weighted')
    return {
        "accuracy": accuracy_score(eval_pred.label_ids, predictions),
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

# Define training arguments
training_args = TrainingArguments(
    output_dir="../data/trec_classification",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    eval_dataset=tokenized_dataset["test"],
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

/var/folders/b4/7fl6nhb926s15j28p0z4y2_40000gn/T/ipykernel_44965/550489574.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.303902,0.112000,0.035730,0.021321,0.112000
2,0.477800,0.295778,0.112000,0.035835,0.021397,0.112000


/Users/ssingla/miniconda3/envs/3.12env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ssingla/miniconda3/envs/3.12env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=682, training_loss=0.47934773240970374, metrics={'train_runtime': 35.3237, 'train_samples_per_second': 308.688, 'train_steps_per_second': 19.307, 'total_flos': 216594249302016.0, 'train_loss': 0.47934773240970374, 'epoch': 2.0})

In [61]:
trainer.evaluate()

/Users/ssingla/miniconda3/envs/3.12env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.295777827501297,
 'eval_accuracy': 0.112,
 'eval_f1': 0.035835346248412635,
 'eval_precision': 0.02139652505251502,
 'eval_recall': 0.112,
 'eval_runtime': 0.7829,
 'eval_samples_per_second': 638.677,
 'eval_steps_per_second': 20.438,
 'epoch': 2.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [70]:
from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA configuration
lora_params = LoraConfig(
    r=8,  
    lora_alpha=32,
    target_modules=['c_attn', 'c_proj'],  
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Apply LoRA modifications to the base model
modified_model = get_peft_model(model, lora_params)
modified_model.print_trainable_parameters()
modified_model.config.pad_token_id = tokenizer.pad_token_id


trainable params: 815,616 || all params: 125,260,032 || trainable%: 0.6511


In [71]:
# Convert class labels for compatibility
processed_dataset = {
    "train": tokenized_dataset["train"].map(lambda x: {"labels": x["label_coarse"]}, batched=True, remove_columns=["label_coarse"]),
    "test": tokenized_dataset["test"].map(lambda x: {"labels": x["label_coarse"]}, batched=True, remove_columns=["label_coarse"])
}

# Format dataset for PyTorch
for split in ["train", "test"]:
    processed_dataset[split].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


In [72]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

# Specify training configurations
train_config = TrainingArguments(
    output_dir="../data/trec_peft_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    save_strategy="epoch",
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./training_logs"
)

# Initialize trainer with dataset and model
model_trainer = Trainer(
    model=modified_model,
    args=train_config,
    compute_metrics=compute_metrics,
    eval_dataset=processed_dataset["test"],
    train_dataset=processed_dataset["train"],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding=True),
)

# Begin training process
model_trainer.train()
modified_model.save_pretrained("optimized_trec_lora")


/Users/ssingla/miniconda3/envs/3.12env/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.397343,0.418000,0.334066,0.449027,0.418000
2,1.835500,1.312817,0.464000,0.388831,0.533090,0.464000


/Users/ssingla/miniconda3/envs/3.12env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ssingla/miniconda3/envs/3.12env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [75]:
from peft import AutoPeftModelForSequenceClassification
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lora_model = (
    AutoPeftModelForSequenceClassification
        .from_pretrained("optimized_trec_lora", num_labels=6, ignore_mismatched_sizes=True)
        .to(device)
)

lora_model.config.pad_token_id = tokenizer.pad_token_id

training_config = TrainingArguments(
    output_dir="../data/trec_peft_analysis",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

model_evaluator = Trainer(
    model=lora_model,
    args=training_config,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)
model_evaluator.evaluate()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/ssingla/miniconda3/envs/3.12env/lib/python3.12/site-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


/Users/ssingla/miniconda3/envs/3.12env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ssingla/miniconda3/envs/3.12env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NameError: name 'evaluation_results' is not defined